In [ ]:
from nidaqmx import Task
from nidaqmx.constants import Edge, AcquisitionType
from nidaqmx.stream_writers import AnalogMultiChannelWriter
import numpy as np
import flammkuchen as fl

import tifffile as tiff
from pathlib import Path
from scipy import fft, signal
import time

In [ ]:
from PyQt5.QtWidgets import QWidget, QApplication, QSlider, QVBoxLayout, QHBoxLayout, QLabel, QPushButton, QFileDialog
from PyQt5 import QtCore
from superqt import QLabeledDoubleSlider

import sys

# Calibration

In [ ]:
class CalibrationWidget(QWidget):
    def __init__(self):
        super().__init__()

        self.dev = "Dev4"
        self.ao_zgalvo = "ao0"# fast galvo
        self.ao_piezo = "ao2" # slow galvo
        
        self.zgalvo_pos = 0
        self.zgalvo_range = [-5, 5]
        
        self.piezo_pos = 0
        self.piezo_range = [-5, 5]
        self.set_task()
        
        self.calibration_points = []
        
        self.set_gui()


    def set_gui(self):

        self.zgalvo_slider = QLabeledDoubleSlider(QtCore.Qt.Horizontal)
        self.zgalvo_slider.setRange(self.zgalvo_range[0], self.zgalvo_range[1])
        self.zgalvo_slider.setValue(0)
        self.zgalvo_slider.valueChanged.connect(self.update)
        zgalvo_layout = QHBoxLayout()
        zgalvo_layout.addWidget(QLabel("Z galvo pos."))
        zgalvo_layout.addWidget(self.zgalvo_slider)
        
        
        self.piezo_slider = QLabeledDoubleSlider(QtCore.Qt.Horizontal)
        self.piezo_slider.setRange(self.piezo_range[0], self.piezo_range[1])
        self.piezo_slider.setValue(0)
        self.piezo_slider.valueChanged.connect(self.update)
        piezo_layout = QHBoxLayout()
        piezo_layout.addWidget(QLabel("Piezo pos."))
        piezo_layout.addWidget(self.piezo_slider)
        
        self.save_button = QPushButton("Store calibration point")
        self.save_button.clicked.connect(self.save_values)
        self.remove_button = QPushButton("Remove last calibration point")
        self.remove_button.clicked.connect(self.remove_values)
        saving_layout = QHBoxLayout()
        saving_layout.addWidget(self.save_button)
        saving_layout.addWidget(self.remove_button)
        
        self.points_label = QLabel("No calibration points yet")
        self.points_label.setWordWrap(True)
        
        self.save_button = QPushButton("Save calibration")
        self.save_button.clicked.connect(self.save_calibration)
        self.save_button.setEnabled(False)

        layout = QVBoxLayout()
        layout.addLayout(zgalvo_layout)
        layout.addLayout(piezo_layout)
        layout.addLayout(saving_layout)
        layout.addWidget(self.points_label)
        layout.addWidget(self.save_button)
        
        self.setLayout(layout)
        
    def save_calibration(self):
        filename = QFileDialog.getSaveFileName(filter="hdf5 files (*.h5)")[0]
        fl.save(filename, self.calibration_points)
    
    def save_values(self):
        self.calibration_points.append([self.zgalvo_pos, self.piezo_pos])
        self.print_points()
        
    def remove_values(self):
        self.calibration_points.pop(-1)
        self.print_points()
        
    def print_points(self):
        if len(self.calibration_points) == 0:
            self.points_label.setText("No calibration points yet")
        else:
            label_text = "{} calibration points:".format(len(self.calibration_points))
            
            for point in self.calibration_points:
                label_text = label_text + "\nZ galvo pos.: {:.3f}, piezo pos.: {:.3f}".format(point[0], point[1])
            self.points_label.setText(label_text)
            
        if len(self.calibration_points) >= 3:
            self.save_button.setEnabled(True)
        else:
            self.save_button.setEnabled(False)
        

    def set_task(self):
        self.cal_task = Task()
        self.cal_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/{self.ao_zgalvo}", min_val=-8, max_val=8)
        self.cal_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/{self.ao_piezo}", min_val=-8, max_val=8)
        self.cal_task.timing.cfg_samp_clk_timing(
            rate=40000,
            source="OnboardClock",
            active_edge=Edge.RISING,
            sample_mode=AcquisitionType.CONTINUOUS,
            samps_per_chan=10000
        )
               
    def update(self, img):

        self.zgalvo_pos = self.zgalvo_slider.value()
        self.piezo_pos = self.piezo_slider.value()
        
        self.cal_task.write([self.zgalvo_pos, self.piezo_pos], auto_start=True)
        

if __name__ == '__main__':
    app = QApplication(sys.argv)
    main = CalibrationWidget()
    main.show()
    sys.exit(app.exec_())


# Run acquisition

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import polyfit

In [ ]:
z_task = Task()
z_task.ao_channels.add_ao_voltage_chan("Dev4/ao0:2", min_val=-10, max_val=10)

z_task.timing.cfg_samp_clk_timing(
    rate=40000,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    samps_per_chan=10000,
)

In [ ]:
xy_task = Task()
xy_task.ao_channels.add_ao_voltage_chan("Dev5/ao0:1", min_val=-10, max_val=10)
xy_task.timing.cfg_samp_clk_timing(
    rate=40000,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    samps_per_chan=10000,
)

In [ ]:
#Load calibration points
calibration_points = fl.load(Path(r"C:\Users\portugueslab\Desktop\calibration.h5"))

galvo_cal_pts = np.array([cal[0] for cal in calibration_points])
piezo_cal_pts = np.array([cal[1] for cal in calibration_points])

#Sort calibration points
ordered_pts = np.argsort(galvo_cal_pts)
galvo_cal_pts = galvo_cal_pts[ordered_pts]
piezo_cal_pts = piezo_cal_pts[ordered_pts]

# Fit line
s, i = polyfit(galvo_cal_pts, piezo_cal_pts, 1)

#Define scanning range (based on Z galvo values)
galvo_scan_range = [-5, 5]

#Generate arrays to write on NI boards
t = np.linspace(0, 1, 10000)

z_galvo = np.linspace(galvo_scan_range[0], galvo_scan_range[1], t.shape[0])
cam = np.zeros_like(t)
piezo = z_galvo*s + i

z_task_arr = np.stack([z_galvo, cam, piezo])

In [ ]:
z_task.write(z_task_arr, auto_start=True)

In [ ]:
z_task.stop()

In [ ]:
plt.figure()
plt.plot(z_galvo)
plt.plot(piezo)

In [ ]:
plt.figure()
plt.imshow(frame)

In [ ]:
#Generate arrays to write on NI boards
t = np.linspace(0, 1, 10000)

### XY scanning to generate light sheet 
xy = signal.sawtooth(2 * np.pi * 100 * t, width=0.5)

# Camera trigger 
cam = np.zeros_like(xy)
cam[1] = 5
cam[3] = -5

# z galvo 
z = signal.sawtooth(2 * np.pi * 1 * t)

# Piezo 
p = signal.sawtooth(2 * np.pi * 1 * t)
# p = np.linspace(5, 0, 100*100 - 1000)
# p = np.append(p, np.linspace(0, 5, 1000))

In [ ]:
plt.figure()
plt.plot(z)
plt.plot(p)

In [ ]:
#Generate arrays to write on NI boards
t = np.linspace(0, 1, 10000)

### XY scanning to generate light sheet 
xy = signal.sawtooth(2 * np.pi * 100 * t, width=0.5)

# Camera trigger 
cam = np.zeros_like(xy)
cam[1] = 5
cam[3] = -5

# z galvo 
z = signal.sawtooth(2 * np.pi * 1 * t)

# Piezo 
p = signal.sawtooth(2 * np.pi * 1 * t)
# p = np.linspace(5, 0, 100*100 - 1000)
# p = np.append(p, np.linspace(0, 5, 1000))

In [ ]:
z.shape

In [ ]:
np.linspace(galvo_scan_range[0], galvo_scan_range[1], t.shape[0])

In [ ]:
t.shape

In [ ]:
# Plotting all the waveforms 
array = np.stack([z, cam, p])
plt.figure()
plt.plot(array.T)
# plt.xlim(0,10000)

In [ ]:
task = Task()
task.ao_channels.add_ao_voltage_chan("Dev4/ao0:2", min_val=-10, max_val=10)

task.timing.cfg_samp_clk_timing(
    rate=40000,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    samps_per_chan=10000,
)

In [ ]:
task2 = Task()
task2.ao_channels.add_ao_voltage_chan("Dev5/ao0:1", min_val=-10, max_val=10)
task2.timing.cfg_samp_clk_timing(
    rate=40000,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    samps_per_chan=10000,
)

In [ ]:
t = np.linspace(0, 1, 500)
xy = signal.sawtooth(2 * np.pi * 5 * t, width=0.5)
plt.plot(xy)

In [ ]:
t = np.linspace(0, 1, 10000)

# Generating a light sheet 
xy = signal.sawtooth(2 * np.pi * 100 * t, width=0.5)

# Camera trigger 
cam = np.zeros_like(xy)
cam[1] = 5
cam[3] = -5

# z galvo 
z = signal.sawtooth(2 * np.pi * 1 * t)

# Piezo 
p = signal.sawtooth(2 * np.pi * 1 * t, width=0) * 5 + 3
# p = np.linspace(5, 0, 100*100 - 1000)
# p = np.append(p, np.linspace(0, 5, 1000))
              
# Plotting all the waveforms 
array = np.stack([z, cam, p])
plt.figure()
plt.plot(array.T)
# plt.xlim(0,10000)
plt.plot(cam)

In [ ]:
task.write(array, auto_start=True)

In [ ]:
array = np.stack([z, cam, p])
task.write(array, auto_start=True)

In [ ]:
array2 = np.stack([xy, xy])

In [ ]:
task2.write(array2, auto_start=True)

In [ ]:
task.stop()

In [ ]:
task2.stop()

In [ ]:
piezo_task = Task()
task.ao_channels.add_ao_voltage_chan("Dev4/ao0:2", min_val=-10, max_val=10)

task.timing.cfg_samp_clk_timing(
    rate=40000,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    samps_per_chan=10000,
)